https://sparkbyexamples.com/pyspark/convert-pandas-to-pyspark-dataframe/

In [1]:
import pandas as pd
from zipfile import ZipFile 
pd.set_option("display.max_columns",1000)

import findspark
import pyspark
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
findspark.init()
#findspark.init('C:\Spark\spark-3.1.1-bin-hadoop2.7')
findspark.find()

'C:\\Spark\\spark-3.1.1-bin-hadoop2.7'

# Create Spark df
ending sesh wipes the tables

In [2]:
spark = SparkSession.builder.appName('Project1').getOrCreate()#.master("local") #?

In [3]:
spark.catalog.listTables()

[]

# BELOW ARE JUST TESTS-----------------------------------------------

## test
"fake" is just a python dataframe....

In [9]:
fake = pd.DataFrame(['poop','pee'])
fake.columns = ['whaterver']
fake

,whaterver
0,poop
1,pee


"sprk" is a Spark dataframe made from our python dataframe

In [10]:
#Create PySpark DataFrame from Pandas
sprk=spark.createDataFrame(fake) 
sprk.printSchema()
sprk.show()

root
 |-- whaterver: string (nullable = true)

+---------+
|whaterver|
+---------+
|     poop|
|      pee|
+---------+



In [11]:
sprk.head()

Row(whaterver='poop')

In [21]:
sprk.select("whaterver").show()

+---------+
|whaterver|
+---------+
|     poop|
|      pee|
+---------+



now we make a temporary SQL table called 'fake' within Spark. now we can write SQL queries in spark to find it

In [22]:
# Register the DataFrame as a SQL temporary view
sprk.createOrReplaceTempView("fake")

sqlDF = spark.sql("""
    SELECT * FROM fake where whaterver like 'po%'
    """)
sqlDF.show()

+---------+
|whaterver|
+---------+
|     poop|
+---------+



## test 2
has to be a sparkdf first to then create a spark table

In [27]:
randf = pd.DataFrame({
    'fku': ['hey','whatever','who cares'],
    'numbs': [4, 5, 6]
})
randf

,fku,numbs
0,hey,4
1,whatever,5
2,who cares,6


In [28]:
#Create PySpark DataFrame from Pandas
fku=spark.createDataFrame(randf) 
fku.printSchema()
fku.show()

root
 |-- fku: string (nullable = true)
 |-- numbs: long (nullable = true)

+---------+-----+
|      fku|numbs|
+---------+-----+
|      hey|    4|
| whatever|    5|
|who cares|    6|
+---------+-----+



In [29]:
# Register the DataFrame as a SQL temporary view
fku.createOrReplaceTempView("fku")

sqlDF = spark.sql("""
    SELECT * FROM fku where numbs >=5
    """)
sqlDF.show()

+---------+-----+
|      fku|numbs|
+---------+-----+
| whatever|    5|
|who cares|    6|
+---------+-----+



# ABOVE ARE JUST TESTS -----------------------------------------------

## Real Deal

### CSV to python df

In [4]:
df = pd.read_csv('flights.csv.zip', nrows=1000)
df = df.fillna(0)
df['DEP_TIME'] = df['DEP_TIME'].astype('int')
df['DEP_HR'] = df['DEP_TIME'].astype('str').str[:-2] 
df['ARR_TIME'] = df['ARR_TIME'].astype('int')
df = df.fillna(0)
df = df.astype('str')
df

,FL_DATE,TAIL_NUM,CARRIER,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 19,DEP_HR
0,2019-01-01,N8974C,9E,AVL,"Asheville, NC",ATL,"Atlanta, GA",1658,-7.0,1758,-22.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16
1,2019-01-01,N922XJ,9E,JFK,"New York, NY",RDU,"Raleigh/Durham, NC",1122,-8.0,1255,-29.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11
2,2019-01-01,N326PQ,9E,CLE,"Cleveland, OH",DTW,"Detroit, MI",1334,-7.0,1417,-31.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
3,2019-01-01,N135EV,9E,BHM,"Birmingham, AL",ATL,"Atlanta, GA",1059,-1.0,1255,-8.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
4,2019-01-01,N914XJ,9E,GTF,"Great Falls, MT",MSP,"Minneapolis, MN",1057,-3.0,1418,-17.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2019-01-01,N257NN,MQ,STL,"St. Louis, MO",ORD,"Chicago, IL",1220,14.0,1327,-7.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12
996,2019-01-01,N855AE,MQ,LGA,"New York, NY",CMH,"Columbus, OH",1048,-12.0,1233,-34.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
997,2019-01-01,N688AE,MQ,ORD,"Chicago, IL",COU,"Columbia, MO",2317,52.0,104,80.0,0.0,0,0.0,0.0,0.0,28.0,0.0,52.0,0.0,23
998,2019-01-01,N262NN,MQ,MSN,"Madison, WI",ORD,"Chicago, IL",0,0.0,0,0.0,1.0,B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,


In [13]:
#df[df['DEP_HR'] == 'null']

,FL_DATE,TAIL_NUM,CARRIER,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 19,DEP_HR


### Define Schema
everything is a string at first ...

In [5]:
from pyspark.sql.types import StructType, IntegerType, DateType
from pyspark.sql.types import *

schema = StructType([
    StructField("FL_DATE", StringType()),#, DateType()),
    StructField("TAIL_NUM", StringType()),
    StructField("CARRIER", StringType()),
    StructField("ORIGIN", StringType()),
    StructField("ORIGIN_CITY_NAME", StringType()),
    StructField("DEST", StringType()),
    StructField("DEST_CITY_NAME", StringType()),
    StructField("DEP_TIME", StringType()),
    StructField("DEP_DELAY", StringType()),#, DoubleType()),
    StructField("ARR_TIME", StringType()),
    StructField("ARR_DELAY", StringType()),#, DoubleType()),
    StructField("CANCELLED", StringType()),
    StructField("CANCELLATION_CODE", StringType()),
    StructField("DIVERTED", StringType()),
    StructField("CARRIER_DELAY", StringType()),
    StructField("WEATHER_DELAY", StringType()),
    StructField("NAS_DELAY", StringType()),
    StructField("SECURITY_DELAY", StringType()),
    StructField("LATE_AIRCRAFT_DELAY", StringType()),
    StructField("Unnamed: 19", StringType()),#, IntegerType())
    StructField("DEP_HR", StringType())
])

### Create Spark DF from pandas df

In [6]:
#Create PySpark DataFrame from Pandas
flts=spark.createDataFrame(df,schema=schema) 
flts.printSchema()
flts.show()

root
 |-- FL_DATE: string (nullable = true)
 |-- TAIL_NUM: string (nullable = true)
 |-- CARRIER: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY_NAME: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_CITY_NAME: string (nullable = true)
 |-- DEP_TIME: string (nullable = true)
 |-- DEP_DELAY: string (nullable = true)
 |-- ARR_TIME: string (nullable = true)
 |-- ARR_DELAY: string (nullable = true)
 |-- CANCELLED: string (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: string (nullable = true)
 |-- CARRIER_DELAY: string (nullable = true)
 |-- WEATHER_DELAY: string (nullable = true)
 |-- NAS_DELAY: string (nullable = true)
 |-- SECURITY_DELAY: string (nullable = true)
 |-- LATE_AIRCRAFT_DELAY: string (nullable = true)
 |-- Unnamed: 19: string (nullable = true)
 |-- DEP_HR: string (nullable = true)

+----------+--------+-------+------+--------------------+----+------------------+--------+---------+-------

### Change Delays from String to Doubles

In [7]:
from pyspark.sql.types import DateType
flts = flts.withColumn("DEP_DELAY", flts['DEP_DELAY'].cast(DoubleType())).withColumn('ARR_DELAY', flts['ARR_DELAY'].cast(DoubleType())).withColumn("DEP_HR", flts['DEP_HR'].cast(DoubleType()))
flts.show()
print(flts.schema)

+----------+--------+-------+------+--------------------+----+------------------+--------+---------+--------+---------+---------+-----------------+--------+-------------+-------------+---------+--------------+-------------------+-----------+------+
|   FL_DATE|TAIL_NUM|CARRIER|ORIGIN|    ORIGIN_CITY_NAME|DEST|    DEST_CITY_NAME|DEP_TIME|DEP_DELAY|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|Unnamed: 19|DEP_HR|
+----------+--------+-------+------+--------------------+----+------------------+--------+---------+--------+---------+---------+-----------------+--------+-------------+-------------+---------+--------------+-------------------+-----------+------+
|2019-01-01|  N8974C|     9E|   AVL|       Asheville, NC| ATL|       Atlanta, GA|    1658|     -7.0|    1758|    -22.0|      0.0|                0|     0.0|          0.0|          0.0|      0.0|           0.0|                0.0|        0.0|  16.0|
|201

In [8]:
flts.createOrReplaceTempView("flights")

# Task 1 [20 points]
Your first task is to calculate the average flight delays in the dataset. Your supervisor made it
clear that you should choose SparkSQL with Python and DataFrames, so that your code should
be compatible with other software products of your company. Your deliverables for this task
are the following:
- A Python file (named “task1.py”) containing the code to produce the desired result.
- A report (named “task1.pdf”) explaining the basic intuition of your code.
- A screenshot (named “task1.png”) of the produced output (e.g., showing the result in
the console). 

## Departure Delay

In [9]:
query = spark.sql("""
    SELECT avg(DEP_DELAY)
    FROM flights 
    """)
query.show()

+--------------+
|avg(DEP_DELAY)|
+--------------+
|         6.596|
+--------------+



## Arrival Delay

In [10]:
query = spark.sql("""
    SELECT avg(ARR_DELAY)
    FROM flights 
    """)
query.show()

+--------------+
|avg(ARR_DELAY)|
+--------------+
|         0.532|
+--------------+



# Task 2 [35 points]
For this task you continue to work with SparkSQL. The objective is to create reports on the
average and median departure delays of 
- (a) all the airports
- (b) all the airways in the dataset
You should give four reports: 
- two for the airports (average/median delays)
- two for the airways (average/median delays)

Each report is a CSV file containing one line for each airport/airway 
and the lines of each file should be ordered (in descending order) based on the
corresponding criterion (average/median delay). 
No header files are required for these files.
An extra instruction you have from your supervisor is that you should take care of some data
outliers: you should not consider in your analysis any airports/airways that have extremely
low number of flights; the criterion is that any airport/airway belonging in the lowest 1%
percentile, regarding the number of flights, should be omitted. Your deliverables for this task
are the following:
- A Python file (named “task2.py”) containing the code to produce the reports.
- A report (named “task2.pdf”) explaining the basic intuition of your code.
- The four report files (named “task2-ap-avg.csv”, “task2-ap-med.csv”, “task2-awavg.csv”, and “task2-aw-med.csv”) having the determined file structure. Please
restrict the number of lin

## Airports Departure Delays

### Average

In [38]:
query = spark.sql("""
    SELECT 
    ORIGIN
    , avg(DEP_DELAY)
    FROM flights 
    group by ORIGIN
    order by 2 desc;
    """)
perc = query.stat.approxQuantile("avg(DEP_DELAY)",[0.01],0.0)
perc = int(perc[0])
perc

-13

In [41]:
query = spark.sql("""
    SELECT ORIGIN
    , avg(DEP_DELAY)
    FROM flights 
    group by ORIGIN
    having avg(DEP_DELAY) > {}
    order by 2 desc
    limit 100;
    """.format(perc))
query.repartition(1).write.csv("task2-ap-avg.csv", sep=',',header=None)
query.show()

+------+------------------+
|ORIGIN|    avg(DEP_DELAY)|
+------+------------------+
|   TVC|             209.0|
|   VPS|             131.5|
|   BIS|             124.0|
|   PWM|              87.5|
|   BIL|              59.0|
|   GTF|              49.0|
|   RFD|              46.0|
|   TUS|              36.0|
|   CMH|34.285714285714285|
|   MSN|              34.0|
|   XNA|              34.0|
|   ELP|              31.5|
|   FWA|              31.0|
|   BLI|30.666666666666668|
|   ABQ|              29.0|
|   KOA|              28.0|
|   ALB|              27.0|
|   AZA|26.333333333333332|
|   ONT|              25.0|
|   MCI|24.666666666666668|
+------+------------------+
only showing top 20 rows



### Median

In [45]:
query = spark.sql("""
    SELECT ORIGIN
    ,percentile(DEP_DELAY, 0.5) as med
    FROM flights 
    group by ORIGIN
    order by 2 desc;
    """)
perc = query.stat.approxQuantile("med",[0.01],0.0)
perc = int(perc[0])
perc

-13

In [47]:
query = spark.sql("""
    SELECT ORIGIN
    ,percentile(DEP_DELAY, 0.5) as med
    FROM flights 
    group by ORIGIN
    having med > {}
    order by 2 desc
    limit 100;
    """.format(perc))
query.repartition(1).write.csv("task2-ap-med.csv", sep=',',header=None)
query.show()

+------+-----+
|ORIGIN|  med|
+------+-----+
|   TVC|209.0|
|   VPS|131.5|
|   BIS|124.0|
|   PWM| 87.5|
|   BIL| 59.0|
|   GTF| 49.0|
|   RFD| 46.0|
|   MSN| 34.0|
|   XNA| 34.0|
|   ELP| 31.5|
|   FWA| 31.0|
|   ABQ| 29.0|
|   KOA| 28.0|
|   ALB| 27.0|
|   ONT| 25.0|
|   SFB| 23.0|
|   COU| 23.0|
|   AZA| 21.0|
|   BQN| 21.0|
|   MSO| 21.0|
+------+-----+
only showing top 20 rows



## Airways Departure Delays

### Average

In [49]:
query = spark.sql("""
    SELECT 
    CARRIER
    , avg(DEP_DELAY)
    FROM flights 
    group by CARRIER
    order by 2 desc;
    """)
perc = query.stat.approxQuantile("avg(DEP_DELAY)",[0.01],0.0)
perc = int(perc[0])
perc

2

In [50]:
query = spark.sql("""
    SELECT
    CARRIER
    , avg(DEP_DELAY)
    FROM flights 
    group by CARRIER
    having avg(DEP_DELAY) > {}
    order by 2 desc
    limit 100;
    """.format(perc))
query.repartition(1).write.csv("task2-aw-avg.csv", sep=',',header=None)
query.show()

+-------+------------------+
|CARRIER|    avg(DEP_DELAY)|
+-------+------------------+
|     G4|11.942857142857143|
|     MQ|  7.91044776119403|
|     AA| 7.249578414839798|
|     NK| 4.504273504273504|
|     9E|2.6405228758169934|
+-------+------------------+



### Median

In [51]:
query = spark.sql("""
    SELECT
    CARRIER
    ,percentile(DEP_DELAY, 0.5) as med
    FROM flights 
    group by CARRIER
    order by 2 desc;
    """)
perc = query.stat.approxQuantile("med",[0.01],0.0)
perc = int(perc[0])
perc

-5

In [52]:
query = spark.sql("""
    SELECT
    CARRIER
    ,percentile(DEP_DELAY, 0.5) as med
    FROM flights 
    group by CARRIER
    having med > {}
    order by 2 desc
    limit 100;
    """.format(perc))
query.repartition(1).write.csv("task2-aw-med.csv", sep=',',header=None)
query.show()

+-------+----+
|CARRIER| med|
+-------+----+
|     MQ| 0.0|
|     AA|-1.0|
|     G4|-1.5|
|     NK|-3.0|
+-------+----+



# Task 3 [45 points]
As a final task, your supervisor assigned to you to investigate if it is possible to train a linear
regression model that could predict the departure delay a flight may have by using, 
as input:
- its origin (column “ORIGIN”)
- its airways (column “CARRIER”)
- its departure time (column “DEP_TIME”)

Again you should use Python and DataFrames, this time with **MLlib**

You should pay attention to transform the string-based input features using the proper representation
format, and you should explain your choices.

Special attention should be given to the “DEP_TIME” column: 
your supervisor told you that you should only consider the corresponding hour of the day 
- (which, of course, should be transformed in a one-hot representation).

For your training and testing workflow you should first remove outliers (see Task 2) 
and then split your dataset into two parts:
- one that will be used for training reasons and it will contain 70% of the entries
- a second one containing the remaining entries andwhich will be used for the assessment of the model. 

No need to implement a more sophisticated evaluation process (e.g., based on k-fold) is required in this phase. 

Your code should:
- (a) prepare the feature vectors
- (b) prepare the training and testing datasets
- (c) train the model
- (d) print on the screen the first 10 predictions, i.e., pairs of feature vectors (in compact format) and predicted outputs, on the screen 
- (e) evaluate the accuracy of the model and display the corresponding metric on the screen. 

Your deliverables are the following:
- A Python file (named “task3.py”) containing the code of the preprocessing, training, and evaluation phase of your machine learning workflow.
- A report (named “task3.pdf”) explaining the basic intuition of your code and your design decisions and assumptions.
- A screenshot (named “task3.png”) showing the output of your machine learning workflow (e.g., showing the results in the console). 

In [9]:
query = spark.sql("""
    SELECT
    ORIGIN
    , CARRIER
    , DEP_HR
    , DEP_DELAY
    FROM flights 
    order by DEP_DELAY desc;
    """)
perc1 = query.stat.approxQuantile("DEP_DELAY",[0.01],0.0)
perc1 = int(perc1[0])
perc99 = query.stat.approxQuantile("DEP_DELAY",[0.99],0.0)
perc99 = int(perc99[0])
print(perc1,' , ',perc99)

-15  ,  110


In [10]:
query = spark.sql("""
    SELECT
    ORIGIN
    , CARRIER
    , DEP_HR
    , DEP_DELAY
    FROM flights
    where DEP_DELAY between {} and {}
    order by DEP_DELAY desc;
    """.format(perc1,perc99))
query.show()

+------+-------+------+---------+
|ORIGIN|CARRIER|DEP_HR|DEP_DELAY|
+------+-------+------+---------+
|   TUS|     AA|  21.0|    110.0|
|   GTF|     G4|  20.0|    101.0|
|   DFW|     AA|  23.0|    101.0|
|   CLT|     AA|  11.0|    100.0|
|   BLI|     G4|  18.0|    100.0|
|   MCI|     AA|   7.0|    100.0|
|   PHX|     AA|  11.0|     96.0|
|   DFW|     AA|  14.0|     95.0|
|   AZA|     G4|  17.0|     94.0|
|   PHL|     AA|  null|     93.0|
|   CLT|     AA|  13.0|     83.0|
|   PHX|     AA|  22.0|     82.0|
|   TPA|     AA|  15.0|     81.0|
|   ORD|     AA|  21.0|     80.0|
|   CLT|     AA|  15.0|     79.0|
|   DFW|     AA|  11.0|     77.0|
|   RSW|     AA|  18.0|     71.0|
|   MIA|     AA|  18.0|     70.0|
|   MSN|     MQ|  19.0|     68.0|
|   ELP|     MQ|  17.0|     67.0|
+------+-------+------+---------+
only showing top 20 rows



## Prepare Feature Vectors

In [11]:
prep = query.na.drop()
#prep.select("DEP_HR").distinct().orderBy("DEP_HR").show()

### Get Indexes

In [12]:
from pyspark.ml.feature import StringIndexer

ORGindexer = StringIndexer()\
.setInputCol("ORIGIN")\
.setOutputCol("ORIGIN_indx")

CARindexer = StringIndexer()\
.setInputCol("CARRIER")\
.setOutputCol("CARRIER_indx")

### One Hot Encode

In [13]:
from pyspark.ml.feature import OneHotEncoder

ORGencoder = OneHotEncoder(dropLast=False)\
.setInputCols(["ORIGIN_indx"])\
.setOutputCols(["ORIGIN_encd"])

CARencoder = OneHotEncoder(dropLast=False)\
.setInputCols(["CARRIER_indx"])\
.setOutputCols(["CARRIER_encd"])

In [14]:
from pyspark.ml.feature import VectorAssembler
vector_assembler = VectorAssembler()\
.setInputCols(["ORIGIN_encd","CARRIER_encd","DEP_HR"])\
.setOutputCol("features")

In [15]:
from pyspark.ml import Pipeline
transformation_pipeline = Pipeline()\
.setStages([ORGindexer, CARindexer, ORGencoder, CARencoder, vector_assembler])
fitted_pipeline = transformation_pipeline.fit(prep)

In [16]:
regr = fitted_pipeline.transform(prep)
regr.cache()
regr.show(5)

+------+-------+------+---------+-----------+------------+----------------+-------------+--------------------+
|ORIGIN|CARRIER|DEP_HR|DEP_DELAY|ORIGIN_indx|CARRIER_indx|     ORIGIN_encd| CARRIER_encd|            features|
+------+-------+------+---------+-----------+------------+----------------+-------------+--------------------+
|   TUS|     AA|  21.0|    110.0|       60.0|         0.0|(144,[60],[1.0])|(5,[0],[1.0])|(150,[60,144,149]...|
|   GTF|     G4|  20.0|    101.0|       71.0|         3.0|(144,[71],[1.0])|(5,[3],[1.0])|(150,[71,147,149]...|
|   DFW|     AA|  23.0|    101.0|        0.0|         0.0| (144,[0],[1.0])|(5,[0],[1.0])|(150,[0,144,149],...|
|   BLI|     G4|  18.0|    100.0|       55.0|         3.0|(144,[55],[1.0])|(5,[3],[1.0])|(150,[55,147,149]...|
|   CLT|     AA|  11.0|    100.0|        2.0|         0.0| (144,[2],[1.0])|(5,[0],[1.0])|(150,[2,144,149],...|
+------+-------+------+---------+-----------+------------+----------------+-------------+--------------------+
o

In [17]:
regr.createOrReplaceTempView("regr")

In [18]:
rquery = spark.sql("""
    SELECT
    features
    , DEP_DELAY as label
    FROM regr
    group by features
    , DEP_DELAY;
    """)
rquery.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(150,[60,144,149]...|110.0|
|(150,[71,147,149]...|101.0|
|(150,[0,144,149],...|101.0|
|(150,[55,147,149]...|100.0|
|(150,[2,144,149],...|100.0|
|(150,[36,144,149]...|100.0|
|(150,[1,144,149],...| 96.0|
|(150,[0,144,149],...| 95.0|
|(150,[39,147,149]...| 94.0|
|(150,[2,144,149],...| 83.0|
|(150,[1,144,149],...| 82.0|
|(150,[20,144,149]...| 81.0|
|(150,[3,144,149],...| 80.0|
|(150,[2,144,149],...| 79.0|
|(150,[0,144,149],...| 77.0|
|(150,[18,144,149]...| 71.0|
|(150,[5,144,149],...| 70.0|
|(150,[126,148,149...| 68.0|
|(150,[67,148,149]...| 67.0|
|(150,[2,144,149],...| 66.0|
+--------------------+-----+
only showing top 20 rows



## Train Test Split

https://stackoverflow.com/questions/40293970/spark-data-frame-random-splitting

In [19]:
split = rquery.randomSplit([0.7, 0.3])
train = split[0]
train.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(150,[0,144,149],...|101.0|
|(150,[36,144,149]...|100.0|
|(150,[55,147,149]...|100.0|
|(150,[60,144,149]...|110.0|
|(150,[71,147,149]...|101.0|
|(150,[0,144,149],...| 95.0|
|(150,[1,144,149],...| 96.0|
|(150,[2,144,149],...| 83.0|
|(150,[39,147,149]...| 94.0|
|(150,[0,144,149],...| 77.0|
|(150,[2,144,149],...| 79.0|
|(150,[3,144,149],...| 80.0|
|(150,[20,144,149]...| 81.0|
|(150,[2,144,149],...| 66.0|
|(150,[5,144,149],...| 70.0|
|(150,[67,148,149]...| 67.0|
|(150,[126,148,149...| 68.0|
|(150,[10,146,149]...| 59.0|
|(150,[16,144,149]...| 58.0|
|(150,[93,144,149]...| 58.0|
+--------------------+-----+
only showing top 20 rows



In [20]:
test = split[1]
test.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(150,[2,144,149],...|100.0|
|(150,[1,144,149],...| 82.0|
|(150,[18,144,149]...| 71.0|
|(150,[3,144,149],...| 65.0|
|(150,[4,144,149],...| 60.0|
|(150,[1,144,149],...| 57.0|
|(150,[62,147,149]...| 57.0|
|(150,[32,144,149]...| 54.0|
|(150,[3,144,149],...| 48.0|
|(150,[0,144,149],...| 45.0|
|(150,[40,144,149]...| 44.0|
|(150,[0,144,149],...| 42.0|
|(150,[12,144,149]...| 43.0|
|(150,[0,148,149],...| 36.0|
|(150,[1,144,149],...| 34.0|
|(150,[1,144,149],...| 33.0|
|(150,[0,144,149],...| 31.0|
|(150,[24,147,149]...| 30.0|
|(150,[0,144,149],...| 29.0|
|(150,[1,144,149],...| 28.0|
+--------------------+-----+
only showing top 20 rows



## Train the model

In [21]:
# Import LinearRegression class
from pyspark.ml.regression import LinearRegression

# Define LinearRegression algorithm
lr = LinearRegression()

# Fit 2 models, using different regularization parameters
model = lr.fit(rquery, {lr.regParam:0.0})
#modelB = lr.fit(data, {lr.regParam:100.0})

## Make predictions

In [22]:
# Make predictions
predictions = model.transform(rquery)
predictions.show(10)

+--------------------+-----+------------------+
|            features|label|        prediction|
+--------------------+-----+------------------+
|(150,[60,144,149]...|110.0| 41.74907470931341|
|(150,[71,147,149]...|101.0| 58.05154187284671|
|(150,[0,144,149],...|101.0|15.721209114835599|
|(150,[55,147,149]...|100.0| 32.15716083133794|
|(150,[2,144,149],...|100.0| 4.510098821539601|
|(150,[36,144,149]...|100.0|23.745331688256393|
|(150,[1,144,149],...| 96.0|10.802946022944319|
|(150,[0,144,149],...| 95.0| 9.972133713762286|
|(150,[39,147,149]...| 94.0| 37.06083788203532|
|(150,[2,144,149],...| 83.0| 5.787671132889225|
+--------------------+-----+------------------+
only showing top 10 rows



## Evalutate the model

In [39]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="rmse")
RMSE = evaluator.evaluate(predictions)
print("Model: Root Mean Squared Error = " + str(RMSE))
# ModelA: Root Mean Squared Error = 128.602026843

Model: Root Mean Squared Error = 15.734990564851342


In [25]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="mae")
MAE = evaluator.evaluate(predictions)
print("Model: Mean Absolute Error = " + str(MAE))

Model: Mean Absolute Error = 9.788315035869392


# Bonus question [+5 points]
Write down something you liked for this assignment and something you disliked. You get all
the points if you write something relevant to the question, regardless of which your opinion
is. If your comments are irrelevant or do not provide any feedback you get no points. Your
deliverable should be a report (named “bonus.pdf”) containing your feedback.

# Submission instructions & honor code
Your code files should be fully replicable and readable (documentation comments are
required and appreciated). Your code should work with Spark v.3 and should be ready to be
executed (e.g., containing all the required import statements). Code failing to execute or
producing wrong results will be penalised. You understand that this is an individual
assignment, and as such you must carry it out alone. You may discuss with your fellow
students to better understand the tasks/questions but you should not ask them to share their
answers with you or to help you by giving you specific advice. 